In [4]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [5]:
first_line = open('../data/hollins.dat', 'r').readline().replace('\n', '')
nodes_count = int(first_line.split(' ')[0])
edges_count = int(first_line.split(' ')[1])

df = pd.read_csv('../data/hollins.dat', sep='\s+', skiprows=1, header=None, names=['NodeID', 'NodeURL'])

df_id_url = df.loc[df.NodeURL.str.contains('http')]
df_id_out_id = df.loc[~df.NodeURL.str.contains('http')]
df_id_out_id.rename(columns={'NodeURL': 'OutNodeID'}, inplace=True)

df_id_out_id['OutNodeID'] = df_id_out_id['OutNodeID'].astype(int)

In [7]:
# import pyspark
# from pyspark.sql import SparkSession

# session = SparkSession.builder.appName('hollins').getOrCreate()
# df_id_out_id_spark = session.createDataFrame(df_id_out_id)

In [8]:
# df_id_out_id.sort_values(by=['NodeID']).to_csv('../data/hollins_id_out_id.csv', index=False, header=False)

In [23]:
df_id_out_id_ranks = df_id_out_id.copy()

# sort by node id
df_id_out_id_ranks = df_id_out_id_ranks.sort_values(by=['NodeID'])

# damping factor
d = 0.25

# initialize ranks
pr = [1/nodes_count for i in range(nodes_count+1)]
pr_2 = pr.copy()

# do iterations
for _ in range(1):
    for i in range(1, nodes_count):
        out_nodes = df_id_out_id_ranks.loc[df_id_out_id_ranks.NodeID == i, 'OutNodeID']

        # go through all out nodes
        for j in out_nodes:
            pr_2[j] += (1-d)/nodes_count + d * (sum([pr[out_node] for out_node in out_nodes]) / out_nodes.count())

    pr = pr_2.copy()

In [26]:
pr_copy = np.array(pr)
# fill nan values with 0
pr_copy[np.isnan(pr_copy)] = 0
print('First 20 liks pages:')
print(pr_copy[0:20])

First 20 liks pages:
[0.00016633 0.00016633 0.13805722 0.00282768 0.00033267 0.00033267
 0.00033267 0.02644711 0.00133067 0.000499   0.00282768 0.00033267
 0.00083167 0.000499   0.00149701 0.000998   0.00166334 0.000499
 0.00365935 0.02611444]


In [38]:
print('Top 5 most likely links:')
sorted_pr = np.argsort(pr_copy)[::-1]
print('\n'.join([f'{df_id_url.loc[df_id_url.NodeID == i].NodeURL.values} {sorted_pr[i]}' for i in sorted_pr[0:5]]))

Top 5 most likely links:
['http://www.hollins.edu/'] 38
['http://www.hollins.edu/admissions/visit/visit.htm'] 430
['http://www.hollins.edu/about/about_tour.htm'] 71
['http://www.hollins.edu/admissions/info-request/info-request.cfm'] 57
['http://www.hollins.edu/htdig/index.html'] 68


In [115]:
# compare outputs
df_outputs_gh = pd.read_csv('../data/output.csv', header=None, names=['NodeID', 'Rank'])
df_outputs_gh.NodeID += 1

In [116]:
df_outputs_gh

,NodeID,Rank
0,1,0.000
1,2,0.021
2,3,0.000
3,4,0.000
4,5,0.000
...,...,...
6007,6008,0.000
6008,6009,0.000
6009,6010,0.000
6010,6011,0.000


In [112]:
df_outputs_networkx = pd.read_csv('../data/result_hollins_id_out_id.csv', header=None, names=['NodeID', 'Rank'])
df_outputs_networkx.sort_values(by=['NodeID'], ascending=True, inplace=True)
df_outputs_networkx

,NodeID,Rank
0,1,0.000058
1,2,0.020210
24,3,0.000113
23,4,0.000060
21,5,0.000060
...,...,...
6006,6008,0.000069
6007,6009,0.000069
6008,6010,0.000069
6010,6011,0.000069
